# Insurance Retention Optimization with Dynamic Pricing

This notebook demonstrates the new insurance pricing module with market cycle support, showing how to optimize retention levels (deductibles) based on dynamic premium calculations.

## Key Features Demonstrated:
1. **Dynamic Premium Calculation** - Based on frequency/severity distributions
2. **Market Cycle Impact** - How hard/soft markets affect optimal retention
3. **Retention Optimization** - Finding the optimal deductible level
4. **Ergodic vs Ensemble Analysis** - Time-average benefits of insurance
5. **Multi-Year Simulation** - Market cycle transitions over time

In [ ]:
# Setup and imports
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent.parent))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple, Any, Optional
import warnings
warnings.filterwarnings('ignore')

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Import insurance modules
from ergodic_insurance.insurance_pricing import (
    InsurancePricer, MarketCycle, PricingParameters, LayerPricing
)
from ergodic_insurance.insurance_program import (
    InsuranceProgram, EnhancedInsuranceLayer, ReinstatementType
)
from ergodic_insurance.loss_distributions import ManufacturingLossGenerator
from ergodic_insurance.manufacturer import WidgetManufacturer
from ergodic_insurance.ergodic_analyzer import ErgodicAnalyzer

print("✅ Modules imported successfully")
print(f"📍 Working directory: {Path.cwd()}")

## 1. Setup Manufacturing Context

First, let's establish our widget manufacturing company context with realistic parameters.

In [ ]:
# Manufacturing company parameters
COMPANY_PARAMS = {
    'initial_assets': 10_000_000,
    'annual_revenue': 15_000_000,
    'base_operating_margin': 0.12,  # 12% margin for viability  
    'asset_turnover': 1.5,
    'tax_rate': 0.25,
    'retention_ratio': 0.7,  # Retain 70% of earnings
}

# Create manufacturer config
from ergodic_insurance.config import ManufacturerConfig

config = ManufacturerConfig(
    initial_assets=COMPANY_PARAMS['initial_assets'],
    base_operating_margin=COMPANY_PARAMS['base_operating_margin'],
    asset_turnover_ratio=COMPANY_PARAMS['asset_turnover'],
    tax_rate=COMPANY_PARAMS['tax_rate'],
    retention_ratio=COMPANY_PARAMS['retention_ratio'],
)

# Create manufacturer
manufacturer = WidgetManufacturer(config)

# Create REALISTIC loss generator with frequency scaling by revenue
# Key insight: Larger businesses have more exposure points (more operations, employees, facilities)
# Frequency should scale with business size

# Base frequencies for a $10M revenue company
BASE_REVENUE = 10_000_000
revenue_scale = COMPANY_PARAMS['annual_revenue'] / BASE_REVENUE

print(f"📊 Revenue Scaling Factor: {revenue_scale:.2f}x")
print(f"   (Frequencies scale with business size)\n")

loss_generator = ManufacturingLossGenerator(
    attritional_params={
        'base_frequency': 3 * revenue_scale,  # More small losses for larger companies
        'severity_mean': 25_000,  # Average $25K per small loss
        'severity_cv': 1.0,  # Moderate variability
    },
    large_params={
        'base_frequency': 0.15 * np.sqrt(revenue_scale),  # Large losses scale sub-linearly
        'severity_mean': 1_200_000,  # Average $1.2M large loss
        'severity_cv': 1.2,  # Higher variability
    },
    catastrophic_params={
        'base_frequency': 0.02 * np.cbrt(revenue_scale),  # Catastrophic events scale slowly
        'severity_alpha': 2.5,  # Moderate tail
        'severity_xm': 3_000_000,  # $3M minimum catastrophic loss
    },
    seed=42,
)

print("🏭 Widget Manufacturer Configuration:")
print(f"  Initial Assets: ${COMPANY_PARAMS['initial_assets']:,.0f}")
print(f"  Annual Revenue: ${COMPANY_PARAMS['annual_revenue']:,.0f}")
print(f"  Base Operating Margin: {COMPANY_PARAMS['base_operating_margin']:.1%}")
print(f"  Expected Annual Profit: ${COMPANY_PARAMS['annual_revenue'] * COMPANY_PARAMS['base_operating_margin']:,.0f}")

# Calculate expected losses with scaled frequencies
print(f"\n📈 Loss Frequency Scaling:")
print(f"  Attritional: {3 * revenue_scale:.1f} events/year (scales linearly with revenue)")
print(f"  Large: {0.15 * np.sqrt(revenue_scale):.2f} events/year (scales with √revenue)")
print(f"  Catastrophic: {0.02 * np.cbrt(revenue_scale):.3f} events/year (scales with ∛revenue)")

# Quick test of loss expectations
test_losses = []
for _ in range(100):
    losses, _ = loss_generator.generate_losses(duration=1.0, revenue=COMPANY_PARAMS['annual_revenue'])
    test_losses.append(sum(l.amount for l in losses))

avg_loss = np.mean(test_losses)
max_loss = np.max(test_losses)
percentile_95 = np.percentile(test_losses, 95)

print(f"\n📊 Expected Loss Profile (100 simulations):")
print(f"  Average Annual Loss: ${avg_loss:,.0f}")
print(f"  95th Percentile Loss: ${percentile_95:,.0f}")
print(f"  Maximum Annual Loss: ${max_loss:,.0f}")
print(f"  Loss Ratio: {avg_loss/COMPANY_PARAMS['annual_revenue']:.1%} of revenue")
print(f"  Profit after avg loss: ${COMPANY_PARAMS['annual_revenue'] * COMPANY_PARAMS['base_operating_margin'] - avg_loss:,.0f}")

# Check if parameters are reasonable
gross_profit = COMPANY_PARAMS['annual_revenue'] * COMPANY_PARAMS['base_operating_margin']
if avg_loss > gross_profit * 0.8:
    print("\n⚠️ Warning: Loss levels may be too high for sustainable operations")
elif avg_loss < gross_profit * 0.2:
    print("\n⚠️ Warning: Loss levels may be too low to justify insurance")
else:
    print("\n✅ Loss parameters appear reasonable for insurance analysis")

## 2. Generate Loss Data for Pricing

Generate synthetic loss data to use for insurance pricing calculations.

In [ ]:
# Generate 10 years of loss data for pricing
n_years = 10
historical_losses = []

for year in range(n_years):
    losses, stats = loss_generator.generate_losses(
        duration=1.0,
        revenue=COMPANY_PARAMS['annual_revenue'],
        include_catastrophic=True
    )
    annual_losses = [loss.amount for loss in losses]
    historical_losses.append(annual_losses)

# Calculate loss statistics
total_losses_per_year = [sum(losses) for losses in historical_losses]
avg_annual_loss = np.mean(total_losses_per_year)
max_annual_loss = np.max(total_losses_per_year)
loss_volatility = np.std(total_losses_per_year)

print(f"📊 Historical Loss Statistics ({n_years} years):")
print(f"  Average Annual Loss: ${avg_annual_loss:,.0f}")
print(f"  Maximum Annual Loss: ${max_annual_loss:,.0f}")
print(f"  Loss Volatility: ${loss_volatility:,.0f}")
print(f"  Loss Ratio: {avg_annual_loss / COMPANY_PARAMS['annual_revenue']:.1%}")

# Visualize loss distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Annual loss totals
ax1.bar(range(1, n_years+1), total_losses_per_year, color='coral', alpha=0.7)
ax1.axhline(avg_annual_loss, color='red', linestyle='--', label=f'Average: ${avg_annual_loss:,.0f}')
ax1.set_xlabel('Year')
ax1.set_ylabel('Total Annual Losses ($)')
ax1.set_title('Annual Loss Experience')
ax1.legend()
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))

# Loss frequency distribution
all_losses = [loss for year_losses in historical_losses for loss in year_losses]
ax2.hist(all_losses, bins=30, color='steelblue', alpha=0.7, edgecolor='black')
ax2.set_xlabel('Individual Loss Amount ($)')
ax2.set_ylabel('Frequency')
ax2.set_title('Loss Severity Distribution')
ax2.set_xscale('log')
ax2.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e3:.0f}K' if x < 1e6 else f'${x/1e6:.1f}M'))

plt.tight_layout()
plt.show()

## 3. Dynamic Insurance Pricing

Now let's use the new pricing module to calculate premiums based on the loss distributions.

In [ ]:
# Create standard insurance structure
def create_insurance_structure(deductible: float) -> List[EnhancedInsuranceLayer]:
    """Create a standard 3-layer insurance structure."""
    return [
        EnhancedInsuranceLayer(
            attachment_point=deductible,
            limit=5_000_000 - deductible,
            base_premium_rate=0.015,  # Initial rate (will be recalculated)
            reinstatements=0,
        ),
        EnhancedInsuranceLayer(
            attachment_point=5_000_000,
            limit=20_000_000,
            base_premium_rate=0.008,
            reinstatements=1,
            reinstatement_type=ReinstatementType.FULL,
        ),
        EnhancedInsuranceLayer(
            attachment_point=25_000_000,
            limit=25_000_000,
            base_premium_rate=0.004,
            reinstatements=2,
            reinstatement_type=ReinstatementType.PRO_RATA,
        ),
    ]

# Price insurance in different market conditions using compare_market_cycles
deductible = 250_000
pricing_results = {}

print("💰 Calculating Insurance Premiums by Market Cycle...")
print("=" * 60)
print("Using compare_market_cycles for consistent pricing\n")

# Create a single pricer with consistent parameters
from ergodic_insurance.insurance_pricing import PricingParameters

base_pricer = InsurancePricer(
    loss_generator=loss_generator,
    market_cycle=MarketCycle.NORMAL,  # Base cycle
    parameters=PricingParameters(
        simulation_years=100,  # More years for stable pricing
        confidence_level=0.95,
        expense_ratio=0.25,
        profit_margin=0.15,
        risk_loading=0.10,
    ),
    seed=42,
)

# Get pricing for all market cycles at once (ensures same pure premium)
layer1_pricing = base_pricer.compare_market_cycles(
    attachment_point=deductible,
    limit=5_000_000 - deductible,
    expected_revenue=COMPANY_PARAMS['annual_revenue'],
)

# Price higher layers with simplified approach
layer2_base_premium = 20_000_000 * 0.001  # 0.1% rate for $20M layer
layer3_base_premium = 25_000_000 * 0.0005  # 0.05% rate for $25M layer

# Store results for each market cycle
for market_cycle in MarketCycle:
    # Get layer 1 pricing from comparison (use string key)
    cycle_key = market_cycle.name  # Convert enum to string key
    layer1_market_premium = layer1_pricing[cycle_key].market_premium
    
    # Apply market adjustments to higher layers
    if market_cycle == MarketCycle.HARD:
        layer2_premium = layer2_base_premium * 1.67  # Same ratio as layer 1
        layer3_premium = layer3_base_premium * 1.67
    elif market_cycle == MarketCycle.SOFT:
        layer2_premium = layer2_base_premium * 0.8
        layer3_premium = layer3_base_premium * 0.8
    else:  # NORMAL
        layer2_premium = layer2_base_premium * 1.43
        layer3_premium = layer3_base_premium * 1.43
    
    # Create layers with calculated premiums
    layers = create_insurance_structure(deductible)
    layers[0].base_premium_rate = layer1_market_premium / layers[0].limit
    layers[1].base_premium_rate = layer2_premium / layers[1].limit
    layers[2].base_premium_rate = layer3_premium / layers[2].limit
    
    # Create program
    program = InsuranceProgram(
        layers=layers,
        deductible=deductible,
        name=f"{market_cycle.name} Market Program",
    )
    
    total_premium = program.calculate_annual_premium()
    
    pricing_results[market_cycle] = {
        'program': program,
        'total_premium': total_premium,
        'layer1_pure': layer1_pricing[cycle_key].pure_premium,
        'layer1_technical': layer1_pricing[cycle_key].technical_premium,
        'layer1_market': layer1_market_premium,
        'pricing_summary': {
            'pure_premium': layer1_pricing[cycle_key].pure_premium,
            'expected_frequency': layer1_pricing[cycle_key].expected_frequency,
            'expected_severity': layer1_pricing[cycle_key].expected_severity,
        }
    }
    
    print(f"{market_cycle.name} Market (Loss Ratio: {market_cycle.value:.0%}):")
    print(f"  Layer 1 Pure Premium: ${layer1_pricing[cycle_key].pure_premium:,.0f} (same for all)")
    print(f"  Layer 1 Market Premium: ${layer1_market_premium:,.0f}")
    print(f"  Total Program Premium: ${total_premium:,.0f}\n")

# Display pricing comparison
print("\n💰 Insurance Premium Comparison by Market Cycle:")
print("=" * 60)

premium_data = []
for cycle, result in pricing_results.items():
    premium = result['total_premium']
    loss_ratio = cycle.value
    base_premium_rate = premium / COMPANY_PARAMS['annual_revenue']
    
    print(f"\n{cycle.name} Market (Loss Ratio: {loss_ratio:.0%}):")
    print(f"  Total Premium: ${premium:,.0f}")
    print(f"  Premium Rate: {base_premium_rate:.2%} of revenue")
    print(f"  Premium/Expected Loss: {premium / avg_annual_loss:.1f}x")
    
    premium_data.append({
        'Market': cycle.name,
        'Premium': premium,
        'Rate': base_premium_rate,
    })

# Calculate and display variation
hard_premium = pricing_results[MarketCycle.HARD]['total_premium']
normal_premium = pricing_results[MarketCycle.NORMAL]['total_premium']
soft_premium = pricing_results[MarketCycle.SOFT]['total_premium']
variation = (hard_premium / soft_premium - 1) * 100

print(f"\n📊 Premium Variation Analysis:")
print(f"  Hard vs Soft Market: {variation:.1f}% difference")
print(f"  Hard Market Premium: ${hard_premium:,.0f}")
print(f"  Normal Market Premium: ${normal_premium:,.0f}")
print(f"  Soft Market Premium: ${soft_premium:,.0f}")

# Check if variation is reasonable
if variation < 20:
    print(f"  ⚠️ Note: Variation may be lower than typical market conditions")
elif variation > 100:
    print(f"  ⚠️ Note: Variation may be higher than typical market conditions")
else:
    print(f"  ✅ Variation is within typical market ranges")

# Visualize premium differences
df_premiums = pd.DataFrame(premium_data)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Bar chart of premiums
colors = {'HARD': '#e74c3c', 'NORMAL': '#f39c12', 'SOFT': '#3498db'}
bars = ax1.bar(df_premiums['Market'], df_premiums['Premium'], 
               color=[colors[m] for m in df_premiums['Market']], alpha=0.7)

# Add value labels on bars
for bar, premium in zip(bars, df_premiums['Premium']):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
            f'${premium:,.0f}', ha='center', va='bottom', fontweight='bold')

ax1.set_ylabel('Annual Premium ($)', fontsize=12)
ax1.set_title('Insurance Premium by Market Cycle', fontsize=14, fontweight='bold')
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.2f}M'))
ax1.grid(axis='y', alpha=0.3)

# Add variation annotation
ax1.text(0.5, 0.95, f'Market Variation: {variation:.1f}%', 
        transform=ax1.transAxes, ha='center', va='top',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5),
        fontsize=11, fontweight='bold')

# Pie chart showing relative differences
market_premiums = [hard_premium, normal_premium, soft_premium]
market_names = ['HARD', 'NORMAL', 'SOFT']
colors_list = ['#e74c3c', '#f39c12', '#3498db']

wedges, texts, autotexts = ax2.pie(market_premiums, labels=market_names, colors=colors_list,
                                    autopct='%1.1f%%', startangle=90)
ax2.set_title('Premium Distribution Across Markets', fontsize=14, fontweight='bold')

# Make percentage text bold
for autotext in autotexts:
    autotext.set_fontweight('bold')
    autotext.set_color('white')

plt.tight_layout()
plt.savefig('../../assets/market_cycles_premium_fixed.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Retention Optimization

Find the optimal retention (deductible) level that maximizes ROE while managing risk.

In [ ]:
def evaluate_retention_level(
    deductible: float,
    market_cycle: MarketCycle,
    n_simulations: int = 100,
    pricer: Optional[InsurancePricer] = None
) -> Dict[str, float]:
    """Evaluate a retention level's performance using the pricing module.
    
    Args:
        deductible: Self-insured retention amount
        market_cycle: Current market cycle
        n_simulations: Number of Monte Carlo simulations for evaluation
        pricer: Optional pre-configured pricer (creates one if not provided)
    """
    
    # Use provided pricer or create a stable one
    if pricer is None:
        pricer = InsurancePricer(
            loss_generator=loss_generator,
            market_cycle=market_cycle,
            parameters=PricingParameters(
                simulation_years=100,  # Stable pricing
                confidence_level=0.95,
                expense_ratio=0.25,
                profit_margin=0.15,
                risk_loading=0.10,
            ),
            seed=42,
        )
    
    # Price each layer using the pricing module
    layers = create_insurance_structure(deductible)
    
    # Price primary layer (deductible to 5M)
    pure_premium_l1, stats_l1 = pricer.calculate_pure_premium(
        attachment_point=deductible,
        limit=layers[0].limit,
        expected_revenue=COMPANY_PARAMS['annual_revenue'],
    )
    technical_premium_l1 = pricer.calculate_technical_premium(pure_premium_l1, layers[0].limit)
    market_premium_l1 = pricer.calculate_market_premium(technical_premium_l1, market_cycle)
    layers[0].base_premium_rate = market_premium_l1 / layers[0].limit
    
    # Price excess layer (5M to 25M) - simplified
    pure_premium_l2, stats_l2 = pricer.calculate_pure_premium(
        attachment_point=5_000_000,
        limit=20_000_000,
        expected_revenue=COMPANY_PARAMS['annual_revenue'],
    )
    if pure_premium_l2 > 0:
        technical_premium_l2 = pricer.calculate_technical_premium(pure_premium_l2, 20_000_000)
        market_premium_l2 = pricer.calculate_market_premium(technical_premium_l2, market_cycle)
        layers[1].base_premium_rate = market_premium_l2 / layers[1].limit
    else:
        # Minimal rate for high excess layer
        layers[1].base_premium_rate = 0.0005 if market_cycle == MarketCycle.HARD else 0.0003
    
    # Price high excess layer (25M to 50M) - minimal pricing
    layers[2].base_premium_rate = 0.0002 if market_cycle == MarketCycle.HARD else 0.0001
    
    # Create program with properly priced layers
    program = InsuranceProgram(
        layers=layers,
        deductible=deductible,
        name=f"{market_cycle.name} Market - ${deductible:,.0f} Retention",
        pricing_enabled=True,
    )
    
    # Store pricing results for transparency
    program.pricing_results = [
        LayerPricing(
            attachment_point=layers[0].attachment_point,
            limit=layers[0].limit,
            pure_premium=pure_premium_l1,
            technical_premium=technical_premium_l1,
            market_premium=market_premium_l1,
            rate_on_line=market_premium_l1 / layers[0].limit if layers[0].limit > 0 else 0,
            expected_frequency=stats_l1['expected_frequency'],
            expected_severity=stats_l1['expected_severity'],
            confidence_interval=stats_l1.get('confidence_interval', (0, 0)),
        ),
        LayerPricing(
            attachment_point=layers[1].attachment_point,
            limit=layers[1].limit,
            pure_premium=pure_premium_l2,
            technical_premium=technical_premium_l2 if pure_premium_l2 > 0 else 0,
            market_premium=market_premium_l2 if pure_premium_l2 > 0 else layers[1].base_premium_rate * layers[1].limit,
            rate_on_line=(market_premium_l2 if pure_premium_l2 > 0 else layers[1].base_premium_rate * layers[1].limit) / layers[1].limit if layers[1].limit > 0 else 0,
            expected_frequency=stats_l2['expected_frequency'],
            expected_severity=stats_l2['expected_severity'],
            confidence_interval=stats_l2.get('confidence_interval', (0, 0)),
        ) if pure_premium_l2 > 0 else None,
        None,  # Simplified for highest layer
    ]
    
    total_premium = program.calculate_annual_premium()
    
    # Run simulations to evaluate performance
    metrics = {
        'net_profits': [],
        'retained_losses': [],
        'insurance_recoveries': [],
    }
    
    for _ in range(n_simulations):
        # Generate annual losses
        losses, _ = loss_generator.generate_losses(
            duration=1.0,
            revenue=COMPANY_PARAMS['annual_revenue'],
        )
        loss_amounts = [loss.amount for loss in losses]
        
        # Process through insurance
        program.reset_annual()
        result = program.process_annual_claims(loss_amounts)
        
        # Calculate net profit
        gross_profit = COMPANY_PARAMS['annual_revenue'] * COMPANY_PARAMS['base_operating_margin']
        retained_loss = result['total_deductible']
        net_profit = gross_profit - retained_loss - total_premium
        
        metrics['net_profits'].append(net_profit)
        metrics['retained_losses'].append(retained_loss)
        metrics['insurance_recoveries'].append(result['total_recovery'])
    
    # Calculate performance metrics
    net_profits = np.array(metrics['net_profits'])
    
    return {
        'deductible': deductible,
        'premium': total_premium,
        'expected_profit': np.mean(net_profits),
        'profit_volatility': np.std(net_profits),
        'downside_risk': np.percentile(net_profits, 5),
        'roe': np.mean(net_profits) / COMPANY_PARAMS['initial_assets'],
        'sharpe_ratio': np.mean(net_profits) / np.std(net_profits) if np.std(net_profits) > 0 else 0,
        'avg_retained': np.mean(metrics['retained_losses']),
        'avg_recovery': np.mean(metrics['insurance_recoveries']),
        'layer1_pure_premium': pure_premium_l1,
        'layer1_market_premium': market_premium_l1,
    }

# Test different retention levels using the pricing module
retention_levels = [100_000, 250_000, 500_000, 1_000_000, 2_000_000]
market_cycle = MarketCycle.NORMAL

print(f"🎯 Optimizing Retention in {market_cycle.name} Market...")
print("Using InsurancePricer for accurate premium calculations")
print("Please wait, running simulations...\n")

# Create a stable pricer for all retention evaluations
optimization_pricer = InsurancePricer(
    loss_generator=loss_generator,
    market_cycle=market_cycle,
    parameters=PricingParameters(
        simulation_years=100,
        confidence_level=0.95,
        expense_ratio=0.25,
        profit_margin=0.15,
        risk_loading=0.10,
    ),
    seed=999,  # Fixed seed for reproducibility
)

optimization_results = []
for deductible in retention_levels:
    result = evaluate_retention_level(
        deductible, 
        market_cycle, 
        n_simulations=100,
        pricer=optimization_pricer
    )
    optimization_results.append(result)
    print(f"✓ Evaluated ${deductible:,.0f} deductible")
    print(f"  Pure Premium (Layer 1): ${result['layer1_pure_premium']:,.0f}")
    print(f"  Market Premium (Layer 1): ${result['layer1_market_premium']:,.0f}")
    print(f"  Total Premium: ${result['premium']:,.0f}")

# Create results DataFrame
df_optimization = pd.DataFrame(optimization_results)
df_optimization['total_cost'] = df_optimization['premium'] + df_optimization['avg_retained']

# Find optimal retention
optimal_idx = df_optimization['sharpe_ratio'].idxmax()
optimal_retention = df_optimization.loc[optimal_idx]

print(f"\n🏆 Optimal Retention: ${optimal_retention['deductible']:,.0f}")
print(f"  Expected ROE: {optimal_retention['roe']:.2%}")
print(f"  Sharpe Ratio: {optimal_retention['sharpe_ratio']:.2f}")
print(f"  Annual Premium: ${optimal_retention['premium']:,.0f}")
print(f"  Premium/Expected Loss Ratio: {optimal_retention['premium']/avg_annual_loss:.2f}x")

In [ ]:
# Visualize optimization results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: ROE vs Retention
ax1 = axes[0, 0]
ax1.plot(df_optimization['deductible'], df_optimization['roe'] * 100, 
         marker='o', linewidth=2, markersize=8, color='green')
ax1.axvline(optimal_retention['deductible'], color='red', linestyle='--', alpha=0.5, label='Optimal')
ax1.set_xlabel('Retention Level ($)', fontsize=11)
ax1.set_ylabel('Expected ROE (%)', fontsize=11)
ax1.set_title('Return on Equity vs Retention', fontsize=12, fontweight='bold')
ax1.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))
ax1.grid(True, alpha=0.3)
ax1.legend()

# Plot 2: Sharpe Ratio vs Retention
ax2 = axes[0, 1]
ax2.plot(df_optimization['deductible'], df_optimization['sharpe_ratio'], 
         marker='s', linewidth=2, markersize=8, color='blue')
ax2.axvline(optimal_retention['deductible'], color='red', linestyle='--', alpha=0.5, label='Optimal')
ax2.set_xlabel('Retention Level ($)', fontsize=11)
ax2.set_ylabel('Sharpe Ratio', fontsize=11)
ax2.set_title('Risk-Adjusted Return vs Retention', fontsize=12, fontweight='bold')
ax2.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))
ax2.grid(True, alpha=0.3)
ax2.legend()

# Plot 3: Cost Structure
ax3 = axes[1, 0]
width = 0.35
x = np.arange(len(retention_levels))
bars1 = ax3.bar(x - width/2, df_optimization['premium']/1e6, width, label='Premium', color='coral')
bars2 = ax3.bar(x + width/2, df_optimization['avg_retained']/1e6, width, label='Avg Retained', color='steelblue')
ax3.set_xlabel('Retention Level ($)', fontsize=11)
ax3.set_ylabel('Annual Cost ($M)', fontsize=11)
ax3.set_title('Insurance Cost Components', fontsize=12, fontweight='bold')
ax3.set_xticks(x)
ax3.set_xticklabels([f'${d/1e6:.1f}M' for d in retention_levels])
ax3.legend()
ax3.grid(axis='y', alpha=0.3)

# Plot 4: Downside Risk
ax4 = axes[1, 1]
ax4.plot(df_optimization['deductible'], df_optimization['downside_risk']/1e6, 
         marker='^', linewidth=2, markersize=8, color='red')
ax4.axhline(0, color='black', linestyle='-', alpha=0.3)
ax4.axvline(optimal_retention['deductible'], color='red', linestyle='--', alpha=0.5, label='Optimal')
ax4.set_xlabel('Retention Level ($)', fontsize=11)
ax4.set_ylabel('5th Percentile Profit ($M)', fontsize=11)
ax4.set_title('Downside Risk (Worst 5% Scenarios)', fontsize=12, fontweight='bold')
ax4.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))
ax4.grid(True, alpha=0.3)
ax4.legend()

plt.tight_layout()
plt.show()

# Display optimization table
print("\n📊 Retention Optimization Results:")
display_cols = ['deductible', 'premium', 'avg_retained', 'total_cost', 'roe', 'sharpe_ratio', 'downside_risk']
df_display = df_optimization[display_cols].copy()
df_display.columns = ['Deductible', 'Premium', 'Avg Retained', 'Total Cost', 'ROE', 'Sharpe', 'Downside Risk']
for col in ['Deductible', 'Premium', 'Avg Retained', 'Total Cost', 'Downside Risk']:
    df_display[col] = df_display[col].apply(lambda x: f'${x:,.0f}')
df_display['ROE'] = df_display['ROE'].apply(lambda x: f'{x:.2%}')
df_display['Sharpe'] = df_display['Sharpe'].apply(lambda x: f'{x:.2f}')
print(df_display.to_string(index=False))

## 5. Market Cycle Impact Analysis

Analyze how optimal retention changes across different market cycles.

In [ ]:
# Compare optimal retention across market cycles using the pricing module
market_comparison = {}
test_deductibles = [250_000, 500_000, 1_000_000]

print("🔄 Analyzing Market Cycle Impact...")
print("Using InsurancePricer for consistent pricing across cycles\n")

# Create pricers for each market cycle with the same configuration
pricers = {}
for cycle in MarketCycle:
    pricers[cycle] = InsurancePricer(
        loss_generator=loss_generator,
        market_cycle=cycle,
        parameters=PricingParameters(
            simulation_years=100,  # Stable pricing
            confidence_level=0.95,
            expense_ratio=0.25,
            profit_margin=0.15,
            risk_loading=0.10,
        ),
        seed=777,  # Same seed for consistency
    )
    print(f"Created pricer for {cycle.name} market (Loss Ratio: {cycle.value:.0%})")

print("\nEvaluating retention levels across market cycles...")
print("-" * 50)

for market_cycle in MarketCycle:
    cycle_results = []
    print(f"\n{market_cycle.name} Market:")
    
    for deductible in test_deductibles:
        result = evaluate_retention_level(
            deductible, 
            market_cycle, 
            n_simulations=50,
            pricer=pricers[market_cycle]
        )
        cycle_results.append(result)
        print(f"  ${deductible:,.0f} retention: Premium=${result['premium']:,.0f}, ROE={result['roe']:.2%}")
    
    df_cycle = pd.DataFrame(cycle_results)
    optimal_idx = df_cycle['sharpe_ratio'].idxmax()
    market_comparison[market_cycle] = {
        'optimal_deductible': df_cycle.loc[optimal_idx, 'deductible'],
        'optimal_roe': df_cycle.loc[optimal_idx, 'roe'],
        'optimal_sharpe': df_cycle.loc[optimal_idx, 'sharpe_ratio'],
        'optimal_premium': df_cycle.loc[optimal_idx, 'premium'],
        'all_results': df_cycle,
    }
    print(f"  ✓ Optimal: ${df_cycle.loc[optimal_idx, 'deductible']:,.0f} (Sharpe: {df_cycle.loc[optimal_idx, 'sharpe_ratio']:.2f})")

# Visualize market cycle comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Premium vs Retention by Market
ax1 = axes[0, 0]
for cycle, data in market_comparison.items():
    df = data['all_results']
    ax1.plot(df['deductible'], df['premium'] / 1000, 
             marker='o', linewidth=2, label=f'{cycle.name} Market', markersize=8)

ax1.set_xlabel('Retention Level ($)', fontsize=11)
ax1.set_ylabel('Annual Premium ($K)', fontsize=11)
ax1.set_title('Insurance Premium vs Retention by Market Cycle', fontsize=12, fontweight='bold')
ax1.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Sharpe Ratio by Market and Retention
ax2 = axes[0, 1]
for cycle, data in market_comparison.items():
    df = data['all_results']
    ax2.plot(df['deductible'], df['sharpe_ratio'], 
             marker='s', linewidth=2, label=f'{cycle.name} Market', markersize=8)

ax2.set_xlabel('Retention Level ($)', fontsize=11)
ax2.set_ylabel('Sharpe Ratio', fontsize=11)
ax2.set_title('Risk-Adjusted Returns Across Market Cycles', fontsize=12, fontweight='bold')
ax2.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))
ax2.legend()
ax2.grid(True, alpha=0.3)

# Plot 3: Optimal Retention by Market
ax3 = axes[1, 0]
markets = list(market_comparison.keys())
optimal_retentions = [market_comparison[m]['optimal_deductible'] for m in markets]
optimal_premiums = [market_comparison[m]['optimal_premium'] for m in markets]
colors = ['#e74c3c', '#f39c12', '#3498db']

bars = ax3.bar([m.name for m in markets], optimal_retentions, color=colors, alpha=0.7)
ax3.set_ylabel('Optimal Retention ($)', fontsize=11)
ax3.set_title('Optimal Retention Level by Market Cycle', fontsize=12, fontweight='bold')
ax3.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))

# Add premium amounts as text on bars
for bar, premium in zip(bars, optimal_premiums):
    height = bar.get_height()
    ax3.text(bar.get_x() + bar.get_width()/2., height,
            f'Premium:\n${premium:,.0f}', ha='center', va='bottom', fontweight='bold')

ax3.grid(axis='y', alpha=0.3)

# Plot 4: Premium Comparison at Optimal Retention
ax4 = axes[1, 1]
market_names = [m.name for m in markets]
optimal_sharpes = [market_comparison[m]['optimal_sharpe'] for m in markets]

bars2 = ax4.bar(market_names, optimal_premiums, color=colors, alpha=0.7)
ax4.set_ylabel('Annual Premium at Optimal Retention ($)', fontsize=11)
ax4.set_title('Premium Cost at Optimal Retention by Market', fontsize=12, fontweight='bold')
ax4.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e3:.0f}K'))

# Add Sharpe ratios as text
for bar, sharpe in zip(bars2, optimal_sharpes):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height,
            f'Sharpe: {sharpe:.2f}', ha='center', va='bottom', fontweight='bold')

ax4.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Summary insights
print("\n" + "=" * 60)
print("💡 Key Insights from Market Cycle Analysis:")
print("=" * 60)

for cycle, data in market_comparison.items():
    print(f"\n{cycle.name} Market (Loss Ratio: {cycle.value:.0%}):")
    print(f"  • Optimal Retention: ${data['optimal_deductible']:,.0f}")
    print(f"  • Annual Premium: ${data['optimal_premium']:,.0f}")
    print(f"  • Expected ROE: {data['optimal_roe']:.2%}")
    print(f"  • Sharpe Ratio: {data['optimal_sharpe']:.2f}")
    print(f"  • Premium/Loss Ratio: {data['optimal_premium']/avg_annual_loss:.2f}x")

# Calculate premium variation
hard_optimal_premium = market_comparison[MarketCycle.HARD]['optimal_premium']
soft_optimal_premium = market_comparison[MarketCycle.SOFT]['optimal_premium']
premium_variation = (hard_optimal_premium / soft_optimal_premium - 1) * 100

print(f"\n📊 Market Impact Summary:")
print(f"  • Premium variation (Hard vs Soft): {premium_variation:.1f}%")
print(f"  • Retention strategy: {'Higher retention in hard markets' if market_comparison[MarketCycle.HARD]['optimal_deductible'] > market_comparison[MarketCycle.SOFT]['optimal_deductible'] else 'Lower retention in hard markets'}")
print(f"  • Risk-return tradeoff: Best Sharpe in {max(market_comparison, key=lambda k: market_comparison[k]['optimal_sharpe']).name} market")

print("\n🎯 Strategic Recommendations:")
print("  1. Adjust retention dynamically based on market conditions")
print("  2. Lock in multi-year contracts during soft markets")
print("  3. Consider self-insurance for higher layers in hard markets")
print("  4. Monitor market indicators for optimal timing of program changes")

## 6. Ergodic Analysis: Time vs Ensemble Averages

Demonstrate how insurance transforms from a cost (ensemble view) to growth enabler (ergodic view).

In [ ]:
def simulate_long_term_growth(
    deductible: float,
    market_cycle: MarketCycle,
    n_years: int = 50,
    with_insurance: bool = True,
    loss_sequence: List[List[float]] = None,
    seed: int = None,
    pricer: Optional[InsurancePricer] = None
) -> np.ndarray:
    """Simulate long-term asset growth with or without insurance.
    
    Uses the InsurancePricer module for accurate premium calculations.
    
    Args:
        deductible: Insurance deductible amount
        market_cycle: Market cycle for pricing
        n_years: Number of years to simulate
        with_insurance: Whether to use insurance
        loss_sequence: Pre-generated loss sequence for consistency
        seed: Random seed for loss generation
        pricer: Optional pre-configured pricer
    
    Returns:
        Array of asset values over time
    """
    
    assets = np.zeros(n_years + 1)
    assets[0] = COMPANY_PARAMS['initial_assets']
    
    # Create loss generator with specific seed if generating new losses
    if loss_sequence is None and seed is not None:
        # Use the same realistic parameters as defined in cell 3
        loss_gen_local = ManufacturingLossGenerator(
            attritional_params={
                'base_frequency': 3 * 1.5,  # Scaled by revenue
                'severity_mean': 25_000,
                'severity_cv': 1.0,
            },
            large_params={
                'base_frequency': 0.15 * np.sqrt(1.5),
                'severity_mean': 1_200_000,
                'severity_cv': 1.2,
            },
            catastrophic_params={
                'base_frequency': 0.02 * np.cbrt(1.5),
                'severity_alpha': 2.5,
                'severity_xm': 3_000_000,
            },
            seed=seed,
        )
    else:
        loss_gen_local = loss_generator
    
    annual_premium = 0
    program = None
    
    if with_insurance:
        # Use provided pricer or create one
        if pricer is None:
            pricer = InsurancePricer(
                loss_generator=loss_generator,
                market_cycle=market_cycle,
                parameters=PricingParameters(
                    simulation_years=100,
                    confidence_level=0.95,
                    expense_ratio=0.25,
                    profit_margin=0.15,
                    risk_loading=0.10,
                ),
                seed=42,
            )
        
        # Price insurance layers properly
        layers = create_insurance_structure(deductible)
        
        # Price primary layer
        pure_premium_l1, stats_l1 = pricer.calculate_pure_premium(
            attachment_point=deductible,
            limit=layers[0].limit,
            expected_revenue=COMPANY_PARAMS['annual_revenue'],
        )
        technical_premium_l1 = pricer.calculate_technical_premium(pure_premium_l1, layers[0].limit)
        market_premium_l1 = pricer.calculate_market_premium(technical_premium_l1, market_cycle)
        layers[0].base_premium_rate = market_premium_l1 / layers[0].limit
        
        # Simplified pricing for higher layers
        layers[1].base_premium_rate = 0.0005 if market_cycle == MarketCycle.HARD else 0.0003
        layers[2].base_premium_rate = 0.0002 if market_cycle == MarketCycle.HARD else 0.0001
        
        # Create program
        program = InsuranceProgram(
            layers=layers,
            deductible=deductible,
            name=f"{market_cycle.name} Market Program",
        )
        
        annual_premium = program.calculate_annual_premium()
        
        # Check if premium is affordable
        gross_profit = COMPANY_PARAMS['annual_revenue'] * COMPANY_PARAMS['base_operating_margin']
        if annual_premium > gross_profit * 0.5:  # Cap at 50% of gross profit
            print(f"  ⚠️ Premium ${annual_premium:,.0f} exceeds affordability threshold")
            print(f"     Capping at 50% of gross profit: ${gross_profit * 0.5:,.0f}")
            annual_premium = gross_profit * 0.5
        else:
            print(f"  ✓ Insurance premium: ${annual_premium:,.0f}/year ({annual_premium/COMPANY_PARAMS['annual_revenue']:.2%} of revenue)")
            print(f"    Primary layer pure premium: ${pure_premium_l1:,.0f}")
            print(f"    Market adjustment: {market_cycle.name} ({market_cycle.value:.0%} LR)")
    
    bankruptcies = 0
    
    for year in range(n_years):
        # Generate profit
        gross_profit = COMPANY_PARAMS['annual_revenue'] * COMPANY_PARAMS['base_operating_margin']
        
        # Get or generate losses for this year
        if loss_sequence is not None:
            # Use pre-generated losses
            year_losses = loss_sequence[year] if year < len(loss_sequence) else []
            total_loss = sum(year_losses)
        else:
            # Generate new losses
            losses, _ = loss_gen_local.generate_losses(
                duration=1.0,
                revenue=COMPANY_PARAMS['annual_revenue'],
            )
            year_losses = [loss.amount for loss in losses]
            total_loss = sum(year_losses)
        
        if with_insurance and program:
            # Process through insurance
            program.reset_annual()
            
            # Process each loss individually through the program
            retained_loss = 0
            for loss_amount in year_losses:
                result = program.process_claim(loss_amount)
                retained_loss += result['deductible_paid']
            
            # Total cost = retained losses + annual premium
            net_loss = retained_loss + annual_premium
            
        else:
            # No insurance - bear all losses
            net_loss = total_loss
        
        # Update assets with growth
        net_income = gross_profit - net_loss
        
        # Add a small growth factor for positive net income
        if net_income > 0:
            growth_factor = 1.03  # 3% growth on positive income
            net_income *= growth_factor
        
        assets[year + 1] = assets[year] + net_income
        
        # Check for bankruptcy (but allow continuation with minimum assets)
        if assets[year + 1] <= 0:
            assets[year + 1] = 100_000  # Minimum survival capital
            bankruptcies += 1
    
    if bankruptcies > 0:
        print(f"    [Warning: {bankruptcies} bankruptcy events]")
    
    return assets

# Generate common loss scenarios for fair comparison
print("🎲 Generating Common Loss Scenarios...")
print("=" * 50)

n_years = 50
n_paths = 100
deductible = 250_000  # Lower deductible for better coverage
market_cycle = MarketCycle.NORMAL

# Pre-generate all loss scenarios (same for both with and without insurance)
all_loss_scenarios = []
for i in range(n_paths):
    path_losses = []
    # Use realistic parameters matching cell 3
    loss_gen_scenario = ManufacturingLossGenerator(
        attritional_params={
            'base_frequency': 3 * 1.5,  # Revenue scaled
            'severity_mean': 25_000,
            'severity_cv': 1.0,
        },
        large_params={
            'base_frequency': 0.15 * np.sqrt(1.5),
            'severity_mean': 1_200_000,
            'severity_cv': 1.2,
        },
        catastrophic_params={
            'base_frequency': 0.02 * np.cbrt(1.5),
            'severity_alpha': 2.5,
            'severity_xm': 3_000_000,
        },
        seed=42 + i,  # Different seed for each path
    )
    
    for year in range(n_years):
        losses, _ = loss_gen_scenario.generate_losses(
            duration=1.0,
            revenue=COMPANY_PARAMS['annual_revenue'],
        )
        path_losses.append([loss.amount for loss in losses])
    
    all_loss_scenarios.append(path_losses)

print(f"✓ Generated {n_paths} loss scenarios over {n_years} years")

# Calculate expected annual losses
all_annual_losses = []
for scenario in all_loss_scenarios:
    for year_losses in scenario:
        all_annual_losses.append(sum(year_losses))

avg_annual_loss = np.mean(all_annual_losses)
max_annual_loss = np.max(all_annual_losses)
percentile_95_loss = np.percentile(all_annual_losses, 95)

print(f"\nLoss Statistics:")
print(f"  Average annual loss: ${avg_annual_loss:,.0f}")
print(f"  95th percentile loss: ${percentile_95_loss:,.0f}")
print(f"  Maximum annual loss: ${max_annual_loss:,.0f}")
print(f"  Loss rate: {avg_annual_loss/COMPANY_PARAMS['annual_revenue']:.2%} of revenue")

# Check viability
gross_profit = COMPANY_PARAMS['annual_revenue'] * COMPANY_PARAMS['base_operating_margin']
print(f"\nViability Check:")
print(f"  Gross profit: ${gross_profit:,.0f}")
print(f"  Profit after avg loss: ${gross_profit - avg_annual_loss:,.0f}")
print(f"  Base ROE (no insurance): {(gross_profit - avg_annual_loss)/COMPANY_PARAMS['initial_assets']:.1%}")

# Create a stable pricer for insurance simulations
ergodic_pricer = InsurancePricer(
    loss_generator=loss_generator,
    market_cycle=market_cycle,
    parameters=PricingParameters(
        simulation_years=100,
        confidence_level=0.95,
        expense_ratio=0.25,
        profit_margin=0.15,
        risk_loading=0.10,
    ),
    seed=888,
)

# Simulate with and without insurance using SAME loss scenarios
print(f"\n🏃 Running {n_paths} parallel simulations...")
print("=" * 50)

paths_with = np.zeros((n_paths, n_years + 1))
paths_without = np.zeros((n_paths, n_years + 1))

print("\nWith Insurance (using InsurancePricer):")
sample_path_with = simulate_long_term_growth(
    deductible, market_cycle, n_years, 
    with_insurance=True, 
    loss_sequence=all_loss_scenarios[0],
    pricer=ergodic_pricer
)

print("\nWithout Insurance:")
sample_path_without = simulate_long_term_growth(
    deductible, market_cycle, n_years, 
    with_insurance=False, 
    loss_sequence=all_loss_scenarios[0]
)

print(f"\nRunning all {n_paths} paths (this may take a moment)...")
for i in range(n_paths):
    # Use the SAME loss scenario for both
    loss_scenario = all_loss_scenarios[i]
    
    # With insurance (suppress output)
    import io
    import contextlib
    
    with contextlib.redirect_stdout(io.StringIO()):
        paths_with[i] = simulate_long_term_growth(
            deductible, market_cycle, n_years, 
            with_insurance=True, 
            loss_sequence=loss_scenario,
            pricer=ergodic_pricer
        )
    
    # Without insurance (suppress output)
    with contextlib.redirect_stdout(io.StringIO()):
        paths_without[i] = simulate_long_term_growth(
            deductible, market_cycle, n_years, 
            with_insurance=False, 
            loss_sequence=loss_scenario
        )

print("✓ Simulations complete")

# Calculate growth rates
def calculate_growth_rate(path):
    """Calculate annualized growth rate."""
    if path[-1] <= 0 or path[0] <= 0:
        return -1.0
    return (path[-1] / path[0]) ** (1/n_years) - 1

growth_with = [calculate_growth_rate(path) for path in paths_with]
growth_without = [calculate_growth_rate(path) for path in paths_without]

# Remove failed paths from growth rate calculation
valid_growth_with = [g for g in growth_with if g > -0.5]
valid_growth_without = [g for g in growth_without if g > -0.5]

# Calculate time and ensemble averages
time_avg_with = np.median(valid_growth_with) if valid_growth_with else -1
time_avg_without = np.median(valid_growth_without) if valid_growth_without else -1
ensemble_avg_with = np.mean(valid_growth_with) if valid_growth_with else -1
ensemble_avg_without = np.mean(valid_growth_without) if valid_growth_without else -1

print("\n📈 Growth Rate Analysis:")
print("=" * 50)
print(f"\nWith Insurance (${deductible:,.0f} deductible):")
print(f"  Valid paths: {len(valid_growth_with)}/{n_paths}")
print(f"  Time Average (Median): {time_avg_with:.2%}")
print(f"  Ensemble Average (Mean): {ensemble_avg_with:.2%}")
print(f"\nWithout Insurance:")
print(f"  Valid paths: {len(valid_growth_without)}/{n_paths}")
print(f"  Time Average (Median): {time_avg_without:.2%}")
print(f"  Ensemble Average (Mean): {ensemble_avg_without:.2%}")

if time_avg_with > time_avg_without:
    print(f"\n🎯 Ergodic Benefit: {(time_avg_with - time_avg_without)*100:.1f} bps")
    print("   Insurance enhances long-term growth by reducing tail risk")
else:
    print(f"\n⚠️ No clear ergodic benefit: {(time_avg_without - time_avg_with)*100:.1f} bps advantage without insurance")
    print("   This suggests insurance premiums may be too high relative to risk")
    
if ensemble_avg_without > ensemble_avg_with:
    print(f"📊 Ensemble Cost: {(ensemble_avg_without - ensemble_avg_with)*100:.1f} bps")
    print("   As expected, insurance has negative expected value")
else:
    print(f"📊 Note: Insurance expected value analysis")
    print("   Premium calibration may need adjustment")

In [ ]:
# Visualize ergodic vs ensemble perspectives
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Sample paths with insurance
ax1 = axes[0, 0]
for i in range(min(10, n_paths)):
    ax1.plot(paths_with[i] / 1e6, alpha=0.3, color='green')
median_with = np.median(paths_with, axis=0)
ax1.plot(median_with / 1e6, 'g-', linewidth=3, label='Median Path')
ax1.set_xlabel('Year', fontsize=11)
ax1.set_ylabel('Assets ($M)', fontsize=11)
ax1.set_title('Asset Growth WITH Insurance', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_ylim(bottom=0)

# Plot 2: Sample paths without insurance
ax2 = axes[0, 1]
for i in range(min(10, n_paths)):
    ax2.plot(paths_without[i] / 1e6, alpha=0.3, color='red')
median_without = np.median(paths_without, axis=0)
ax2.plot(median_without / 1e6, 'r-', linewidth=3, label='Median Path')
ax2.set_xlabel('Year', fontsize=11)
ax2.set_ylabel('Assets ($M)', fontsize=11)
ax2.set_title('Asset Growth WITHOUT Insurance', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(bottom=0)

# Plot 3: Growth rate distribution
ax3 = axes[1, 0]
bins = np.linspace(min(min(valid_growth_with, default=0), min(valid_growth_without, default=0)),
                   max(max(valid_growth_with, default=0), max(valid_growth_without, default=0)), 30)
ax3.hist(valid_growth_with, bins=bins, alpha=0.5, color='green', label='With Insurance', density=True)
ax3.hist(valid_growth_without, bins=bins, alpha=0.5, color='red', label='Without Insurance', density=True)
ax3.axvline(time_avg_with, color='green', linestyle='--', linewidth=2, label=f'Time Avg (With): {time_avg_with:.2%}')
ax3.axvline(time_avg_without, color='red', linestyle='--', linewidth=2, label=f'Time Avg (Without): {time_avg_without:.2%}')
ax3.set_xlabel('Annualized Growth Rate', fontsize=11)
ax3.set_ylabel('Density', fontsize=11)
ax3.set_title('Distribution of Growth Rates', fontsize=12, fontweight='bold')
ax3.legend(fontsize=9)
ax3.grid(True, alpha=0.3)

# Plot 4: Direct comparison - median paths
ax4 = axes[1, 1]
ax4.plot(median_with / 1e6, 'g-', linewidth=3, label='With Insurance (Median)', alpha=0.8)
ax4.plot(median_without / 1e6, 'r-', linewidth=3, label='Without Insurance (Median)', alpha=0.8)
ax4.fill_between(range(n_years + 1), 
                  np.percentile(paths_with, 25, axis=0) / 1e6,
                  np.percentile(paths_with, 75, axis=0) / 1e6,
                  alpha=0.2, color='green', label='With Ins. (25-75%ile)')
ax4.fill_between(range(n_years + 1),
                  np.percentile(paths_without, 25, axis=0) / 1e6,
                  np.percentile(paths_without, 75, axis=0) / 1e6,
                  alpha=0.2, color='red', label='Without Ins. (25-75%ile)')
ax4.set_xlabel('Year', fontsize=11)
ax4.set_ylabel('Assets ($M)', fontsize=11)
ax4.set_title('Median Path Comparison', fontsize=12, fontweight='bold')
ax4.legend(fontsize=9)
ax4.grid(True, alpha=0.3)
ax4.set_ylim(bottom=0)

plt.tight_layout()
plt.show()

# Calculate survival statistics
bankruptcy_threshold = 1_000_000
survived_with = np.sum(paths_with[:, -1] > bankruptcy_threshold) / n_paths
survived_without = np.sum(paths_without[:, -1] > bankruptcy_threshold) / n_paths

print(f"\n💰 Survival Analysis (Assets > ${bankruptcy_threshold/1e6:.1f}M after {n_years} years):")
print(f"  With Insurance: {survived_with:.1%} survived")
print(f"  Without Insurance: {survived_without:.1%} survived")
if survived_with > survived_without:
    print(f"  Survival Improvement: {(survived_with - survived_without)*100:.1f} percentage points")
else:
    print(f"  ⚠️ Insurance reduces survival: {(survived_without - survived_with)*100:.1f} percentage points advantage without")

# Detailed analysis of insurance effectiveness
print(f"\n📊 Insurance Effectiveness Analysis:")
print("=" * 50)

# Get the annual premium from the first simulation
layers = create_insurance_structure(deductible)
test_program = InsuranceProgram.create_with_pricing(
    layers=layers,
    loss_generator=loss_generator,
    expected_revenue=COMPANY_PARAMS['annual_revenue'],
    market_cycle=market_cycle,
    deductible=deductible,
)
actual_premium = test_program.calculate_annual_premium()

print(f"Annual Premium: ${actual_premium:,.0f}")
print(f"Premium Rate: {actual_premium/COMPANY_PARAMS['annual_revenue']:.2%} of revenue")
print(f"Average Annual Loss: ${avg_annual_loss:,.0f}")
print(f"Premium/Loss Ratio: {actual_premium/avg_annual_loss:.2f}x")

if actual_premium > avg_annual_loss * 1.5:
    print("\n⚠️ WARNING: Premium appears excessive relative to expected losses.")
    print("   Consider adjusting market cycle or loss parameters.")

# Calculate break-even analysis
gross_profit = COMPANY_PARAMS['annual_revenue'] * COMPANY_PARAMS['base_operating_margin']
affordable_premium = gross_profit * 0.3  # Max 30% of profit for insurance
print(f"\nAffordability Analysis:")
print(f"  Gross Profit: ${gross_profit:,.0f}")
print(f"  Affordable Premium (30% of profit): ${affordable_premium:,.0f}")
print(f"  Actual Premium: ${actual_premium:,.0f}")
if actual_premium > affordable_premium:
    print(f"  ❌ Premium exceeds affordability by ${actual_premium - affordable_premium:,.0f}")
else:
    print(f"  ✅ Premium is affordable with ${affordable_premium - actual_premium:,.0f} margin")

# Calculate the insurance value proposition
print(f"\n📈 Insurance Value Proposition:")
print("=" * 50)
median_final_with = np.median(paths_with[:, -1])
median_final_without = np.median(paths_without[:, -1])
mean_final_with = np.mean(paths_with[:, -1])
mean_final_without = np.mean(paths_without[:, -1])

print(f"Median Final Wealth:")
print(f"  With Insurance: ${median_final_with:,.0f}")
print(f"  Without Insurance: ${median_final_without:,.0f}")
print(f"  Difference: ${median_final_with - median_final_without:,.0f}")

print(f"\nMean Final Wealth:")
print(f"  With Insurance: ${mean_final_with:,.0f}")
print(f"  Without Insurance: ${mean_final_without:,.0f}")
print(f"  Difference: ${mean_final_with - mean_final_without:,.0f}")

# Tail risk analysis
percentile_5_with = np.percentile(paths_with[:, -1], 5)
percentile_5_without = np.percentile(paths_without[:, -1], 5)

print(f"\n5th Percentile (Tail Risk):")
print(f"  With Insurance: ${percentile_5_with:,.0f}")
print(f"  Without Insurance: ${percentile_5_without:,.0f}")
print(f"  Risk Reduction: ${percentile_5_with - percentile_5_without:,.0f}")

if median_final_with > median_final_without:
    print("\n✅ ERGODIC BENEFIT CONFIRMED: Insurance enhances time-average growth")
else:
    print("\n⚠️ NO ERGODIC BENEFIT: Consider adjusting insurance structure or loss parameters")

## 7. Multi-Year Market Cycle Simulation

Simulate how market cycles affect long-term insurance costs and optimal retention strategies.

In [ ]:
# Simulate market cycle transitions with dynamic premium adjustments
import random

# Set up initial conditions
current_cycle = MarketCycle.NORMAL
deductible = 500_000
years = 20
random.seed(42)

# Transition probabilities (from insurance.yaml)
transition_probs = {
    MarketCycle.HARD: {MarketCycle.HARD: 0.6, MarketCycle.NORMAL: 0.35, MarketCycle.SOFT: 0.05},
    MarketCycle.NORMAL: {MarketCycle.HARD: 0.2, MarketCycle.NORMAL: 0.6, MarketCycle.SOFT: 0.2},
    MarketCycle.SOFT: {MarketCycle.HARD: 0.05, MarketCycle.NORMAL: 0.35, MarketCycle.SOFT: 0.6},
}

print("📊 Multi-Year Market Cycle Simulation")
print("=" * 60)
print("Using InsurancePricer with stable configuration\n")

# Create a SINGLE pricer with consistent parameters for ALL years
# This ensures consistent pure premium calculations
from ergodic_insurance.insurance_pricing import PricingParameters

stable_pricer = InsurancePricer(
    loss_generator=loss_generator,
    market_cycle=MarketCycle.NORMAL,  # Will be overridden per year
    parameters=PricingParameters(
        simulation_years=200,  # Many years for very stable pricing
        confidence_level=0.95,
        expense_ratio=0.25,
        profit_margin=0.15,
        risk_loading=0.10,
    ),
    seed=12345,  # Fixed seed for reproducibility
)

# Pre-calculate pure premium ONCE for the layer (attachment 500K, limit 4.5M)
print("Calculating stable pure premium (one-time calculation)...")
pure_premium_500k, stats_500k = stable_pricer.calculate_pure_premium(
    attachment_point=deductible,
    limit=5_000_000 - deductible,
    expected_revenue=COMPANY_PARAMS['annual_revenue'],
)
technical_premium_500k = stable_pricer.calculate_technical_premium(
    pure_premium_500k, 
    5_000_000 - deductible
)

print(f"  Pure Premium (500K xs 4.5M): ${pure_premium_500k:,.0f}")
print(f"  Technical Premium: ${technical_premium_500k:,.0f}")
print(f"  Expected Frequency: {stats_500k['expected_frequency']:.3f}")
print(f"  Expected Severity: ${stats_500k['expected_severity']:,.0f}\n")

# Calculate market premiums for each cycle using the SAME pure/technical premium
market_premiums = {}
for cycle in MarketCycle:
    market_premium = stable_pricer.calculate_market_premium(technical_premium_500k, cycle)
    market_premiums[cycle] = market_premium
    print(f"  {cycle.name} Market Premium: ${market_premium:,.0f} (LR: {cycle.value:.0%})")

variation = (market_premiums[MarketCycle.HARD] / market_premiums[MarketCycle.SOFT] - 1) * 100
print(f"\n  Market Variation: {variation:.1f}%")

# Simulate market cycles and calculate premiums
cycle_results = []

print(f"\nSimulating {years} years of market transitions...")
print("-" * 40)

for year in range(years):
    # Create insurance program using the pricing module
    layers = create_insurance_structure(deductible)
    
    # Use the pre-calculated market premium for the current cycle
    layer1_premium = market_premiums[current_cycle]
    
    # Apply simple adjustments to higher layers based on market cycle
    if current_cycle == MarketCycle.HARD:
        layer2_premium = 20_000_000 * 0.0015  # Higher rate in hard market
        layer3_premium = 25_000_000 * 0.0008
    elif current_cycle == MarketCycle.SOFT:
        layer2_premium = 20_000_000 * 0.0007  # Lower rate in soft market
        layer3_premium = 25_000_000 * 0.0003
    else:  # NORMAL
        layer2_premium = 20_000_000 * 0.001
        layer3_premium = 25_000_000 * 0.0005
    
    # Update layer rates
    layers[0].base_premium_rate = layer1_premium / layers[0].limit
    layers[1].base_premium_rate = layer2_premium / layers[1].limit
    layers[2].base_premium_rate = layer3_premium / layers[2].limit
    
    # Create program with updated pricing
    program = InsuranceProgram(
        layers=layers,
        deductible=deductible,
        name=f"Year {year+1} - {current_cycle.name} Market",
        pricing_enabled=True,
    )
    
    # Store the pricing results
    program.pricing_results = [
        LayerPricing(
            attachment_point=layers[0].attachment_point,
            limit=layers[0].limit,
            pure_premium=pure_premium_500k,
            technical_premium=technical_premium_500k,
            market_premium=layer1_premium,
            rate_on_line=layer1_premium / layers[0].limit if layers[0].limit > 0 else 0,
            expected_frequency=stats_500k['expected_frequency'],
            expected_severity=stats_500k['expected_severity'],
            confidence_interval=stats_500k.get('confidence_interval', (0, 0)),
        ),
        None,  # Simplified pricing for layer 2
        None,  # Simplified pricing for layer 3
    ]
    
    annual_premium = program.calculate_annual_premium()
    
    # Store results
    cycle_results.append({
        'year': year,
        'market_cycle': current_cycle.name,
        'loss_ratio': current_cycle.value,
        'total_premium': annual_premium,
    })
    
    # Print year details for first few years
    if year < 3:
        print(f"Year {year+1}: {current_cycle.name} Market - Premium: ${annual_premium:,.0f}")
    
    # Transition to next market cycle
    probs = transition_probs[current_cycle]
    rand_val = random.random()
    cumulative = 0.0
    
    for next_cycle, prob in probs.items():
        cumulative += prob
        if rand_val <= cumulative:
            if next_cycle != current_cycle and year < years - 1:
                if year < 3:
                    print(f"  → Transitioning to {next_cycle.name} market")
            current_cycle = next_cycle
            break

if years > 3:
    print(f"... ({years - 3} more years)")

# Convert to DataFrame for analysis
df_cycles = pd.DataFrame(cycle_results)
df_cycles['year'] = df_cycles['year'] + 1

# Calculate cumulative premium
df_cycles['cumulative_premium'] = df_cycles['total_premium'].cumsum()

# Visualize market cycle transitions
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# Plot 1: Market cycle states
ax1 = axes[0]
cycle_map = {'HARD': 2, 'NORMAL': 1, 'SOFT': 0}
cycle_values = [cycle_map[c] for c in df_cycles['market_cycle']]
colors_map = {'HARD': '#e74c3c', 'NORMAL': '#f39c12', 'SOFT': '#3498db'}
colors = [colors_map[c] for c in df_cycles['market_cycle']]

ax1.scatter(df_cycles['year'], cycle_values, c=colors, s=100, alpha=0.7)
for i in range(len(df_cycles) - 1):
    ax1.plot([df_cycles['year'].iloc[i], df_cycles['year'].iloc[i+1]], 
             [cycle_values[i], cycle_values[i+1]], 'k-', alpha=0.3)

ax1.set_yticks([0, 1, 2])
ax1.set_yticklabels(['SOFT', 'NORMAL', 'HARD'])
ax1.set_xlabel('Year', fontsize=11)
ax1.set_title('Market Cycle Evolution', fontsize=12, fontweight='bold')
ax1.grid(axis='x', alpha=0.3)

# Add legend for market states
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#3498db', label='SOFT (80% LR)'),
    Patch(facecolor='#f39c12', label='NORMAL (70% LR)'),
    Patch(facecolor='#e74c3c', label='HARD (60% LR)')
]
ax1.legend(handles=legend_elements, loc='upper right')

# Plot 2: Annual premium
ax2 = axes[1]
bars = ax2.bar(df_cycles['year'], df_cycles['total_premium'] / 1e6, 
               color=colors, alpha=0.7, edgecolor='black', linewidth=0.5)

# Add horizontal reference lines for each market level
hard_level = market_premiums[MarketCycle.HARD] + 30_000 + 20_000  # Include higher layers
normal_level = market_premiums[MarketCycle.NORMAL] + 20_000 + 12_500
soft_level = market_premiums[MarketCycle.SOFT] + 14_000 + 7_500

ax2.axhline(hard_level / 1e6, color='#e74c3c', 
            linestyle=':', alpha=0.5, linewidth=2, label='Hard Market Level')
ax2.axhline(normal_level / 1e6, color='#f39c12', 
            linestyle=':', alpha=0.5, linewidth=2, label='Normal Market Level')
ax2.axhline(soft_level / 1e6, color='#3498db', 
            linestyle=':', alpha=0.5, linewidth=2, label='Soft Market Level')

ax2.set_xlabel('Year', fontsize=11)
ax2.set_ylabel('Annual Premium ($M)', fontsize=11)
ax2.set_title('Annual Insurance Premium by Market Cycle\n(Calculated using InsurancePricer)', 
             fontsize=12, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)

# Add average line
avg_premium = df_cycles['total_premium'].mean()
ax2.axhline(avg_premium / 1e6, color='black', linestyle='--', 
            label=f'20-Year Average: ${avg_premium/1e6:.2f}M', alpha=0.7, linewidth=2)
ax2.legend(loc='best', fontsize=9)

# Plot 3: Cumulative premium
ax3 = axes[2]
ax3.plot(df_cycles['year'], df_cycles['cumulative_premium'] / 1e6, 
         linewidth=3, color='purple', marker='o', markersize=6)
ax3.fill_between(df_cycles['year'], 0, df_cycles['cumulative_premium'] / 1e6, 
                  alpha=0.3, color='purple')
ax3.set_xlabel('Year', fontsize=11)
ax3.set_ylabel('Cumulative Premium ($M)', fontsize=11)
ax3.set_title('Cumulative Insurance Cost Over Time', fontsize=12, fontweight='bold')
ax3.grid(True, alpha=0.3)

# Add text showing total cost
total_cost = df_cycles['cumulative_premium'].iloc[-1]
ax3.text(df_cycles['year'].iloc[-1] - 0.5, total_cost / 1e6 + 0.5, 
         f'20-Year Total: ${total_cost/1e6:.1f}M', 
         ha='right', va='bottom', fontweight='bold', fontsize=12,
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.tight_layout()
plt.savefig('../../assets/market_cycles_simulation_fixed.png', dpi=150, bbox_inches='tight')
plt.show()

# Summary statistics
print("\n" + "=" * 60)
print("📊 Market Cycle Simulation Summary (20 Years):")
print("=" * 60)
print(f"\nMarket Cycle Distribution:")
cycle_counts = df_cycles['market_cycle'].value_counts()
for cycle in ['HARD', 'NORMAL', 'SOFT']:
    count = cycle_counts.get(cycle, 0)
    print(f"  {cycle}: {count} years ({count/20:.0%})")

print(f"\nPremium Statistics:")
print(f"  Minimum Annual: ${df_cycles['total_premium'].min():,.0f}")
print(f"  Maximum Annual: ${df_cycles['total_premium'].max():,.0f}")
print(f"  Average Annual: ${df_cycles['total_premium'].mean():,.0f}")
print(f"  Total 20-Year Cost: ${df_cycles['cumulative_premium'].iloc[-1]:,.0f}")
print(f"  Premium Std Dev: ${df_cycles['total_premium'].std():,.0f}")
print(f"  Coefficient of Variation: {df_cycles['total_premium'].std() / df_cycles['total_premium'].mean():.1%}")

# Calculate average premium by market cycle
print(f"\nAverage Premium by Market Cycle:")
for cycle in ['HARD', 'NORMAL', 'SOFT']:
    cycle_premiums = df_cycles[df_cycles['market_cycle'] == cycle]['total_premium']
    if len(cycle_premiums) > 0:
        avg = cycle_premiums.mean()
        print(f"  {cycle}: ${avg:,.0f} ({len(cycle_premiums)} years)")
        
# Market cycle transition analysis
print(f"\nMarket Cycle Transitions:")
transitions = {}
for i in range(len(df_cycles) - 1):
    from_cycle = df_cycles.iloc[i]['market_cycle']
    to_cycle = df_cycles.iloc[i+1]['market_cycle']
    key = f"{from_cycle} → {to_cycle}"
    transitions[key] = transitions.get(key, 0) + 1

for key, count in sorted(transitions.items()):
    print(f"  {key}: {count} times")

print(f"\n💡 Key Insights:")
print("  • The InsurancePricer module provides consistent pricing by market cycle")
print("  • Hard markets command ~{:.0f}% higher premiums than soft markets".format(variation))
print("  • Market transitions drive premium volatility")
print("  • Multi-year contracts during soft markets could lock in savings")
print("  • The pricing module ensures actuarially sound premiums based on loss distributions")

## 8. Key Insights and Recommendations

Summarize the findings and provide actionable recommendations.

In [ ]:
# Generate executive summary
print("🎯 EXECUTIVE SUMMARY: RETENTION OPTIMIZATION WITH DYNAMIC PRICING")
print("=" * 70)

print("\n📌 KEY FINDINGS:")
print("-" * 50)

print("\n1. OPTIMAL RETENTION LEVELS:")
for cycle, data in market_comparison.items():
    print(f"   • {cycle.name} Market: ${data['optimal_deductible']:,.0f} deductible")
    print(f"     Expected ROE: {data['optimal_roe']:.2%}, Sharpe: {data['optimal_sharpe']:.2f}")

print("\n2. PRICING DYNAMICS:")
# Use the actual variation calculated in cell 7
hard_premium = pricing_results[MarketCycle.HARD]['total_premium']
soft_premium = pricing_results[MarketCycle.SOFT]['total_premium']
actual_variation = (hard_premium / soft_premium - 1) * 100

print(f"   • Premium range across cycles: {actual_variation:.1f}% variation")
print(f"   • Hard market premium: {hard_premium / avg_annual_loss:.1f}x expected losses")
print(f"   • Soft market premium: {soft_premium / avg_annual_loss:.1f}x expected losses")
print(f"   • Normal market premium: {pricing_results[MarketCycle.NORMAL]['total_premium'] / avg_annual_loss:.1f}x expected losses")

print("\n3. ERGODIC BENEFITS:")
# Use the actual calculated values from the ergodic analysis section
if 'time_avg_with' in locals() and 'time_avg_without' in locals():
    benefit = (time_avg_with - time_avg_without) * 100
    if benefit > 0:
        print(f"   • Time-average growth improvement: {benefit:.1f} bps")
    else:
        print(f"   • No clear time-average benefit (insurance may be overpriced)")
else:
    print(f"   • Time-average growth: [Run Section 6 for results]")
    
if 'survived_with' in locals() and 'survived_without' in locals():
    survival_diff = (survived_with - survived_without) * 100
    if survival_diff > 0:
        print(f"   • Survival rate improvement: {survival_diff:.1f} percentage points")
    else:
        print(f"   • No survival benefit (consider adjusting parameters)")
else:
    print(f"   • Survival rate: [Run Section 6 for results]")
    
if 'median_final_with' in locals() and 'median_final_without' in locals():
    wealth_ratio = median_final_with / median_final_without
    print(f"   • Median final wealth ratio: {wealth_ratio:.2f}x")
    if wealth_ratio < 1:
        print(f"     ⚠️ Insurance reduces median wealth (premium too high)")
else:
    print(f"   • Final wealth comparison: [Run Section 6 for results]")

# Add note about insurance expected value
if 'ensemble_avg_with' in locals() and 'ensemble_avg_without' in locals():
    if ensemble_avg_with > ensemble_avg_without:
        print(f"\n   ⚠️ NOTE: Insurance shows positive expected value, which is unrealistic.")
        print(f"      This suggests the pricing model needs calibration.")

print("\n4. MARKET CYCLE IMPACT (20-year simulation):")
if 'df_cycles' in locals():
    print(f"   • Average annual premium: ${df_cycles['total_premium'].mean():,.0f}")
    print(f"   • Premium volatility: ±{df_cycles['total_premium'].std() / df_cycles['total_premium'].mean():.0%}")
    print(f"   • Total insurance cost: ${df_cycles['cumulative_premium'].iloc[-1]:,.0f}")
else:
    print(f"   • [Run Section 7 for market cycle simulation results]")

print("\n💡 STRATEGIC RECOMMENDATIONS:")
print("-" * 50)

recommendations = [
    "1. DYNAMIC RETENTION STRATEGY:",
    "   Adjust retention levels based on market cycles:",
    "   - Hard markets → Increase retention to offset higher premiums",
    "   - Soft markets → Lower retention to take advantage of cheap coverage",
    "",
    "2. MULTI-YEAR CONTRACTS:",
    "   Lock in favorable terms during soft markets with 3-5 year agreements",
    "   to smooth premium volatility across cycles",
    "",
    "3. RISK-ADJUSTED OPTIMIZATION:",
]

# Add specific recommendations based on available data
if 'optimal_retention' in locals():
    recommendations.extend([
        f"   Target Sharpe ratio of {optimal_retention['sharpe_ratio']:.2f} or higher",
        f"   Maintain retention around ${optimal_retention['deductible']:,.0f}",
    ])
else:
    recommendations.extend([
        "   Target Sharpe ratio of 2.0 or higher",
        "   Optimize retention based on loss profile",
    ])

recommendations.extend([
    "",
    "4. ERGODIC PERSPECTIVE:",
    "   Focus on time-average growth, not just expected values",
    "   Insurance transforms from cost to growth enabler over long horizons",
    "   However, ensure premiums are reasonable relative to expected losses",
    "",
    "5. CONTINUOUS MONITORING:",
    "   Review retention levels annually",
    "   Track market cycle indicators",
    "   Update loss distributions quarterly",
    "   Validate pricing models against actual experience",
])

for rec in recommendations:
    print(rec)

print("\n📈 EXPECTED OUTCOMES WITH OPTIMIZATION:")
print("-" * 50)

# Calculate improvements based on available data
baseline_roe = 0.08  # 8% baseline

if 'optimal_retention' in locals():
    print(f"• ROE improvement: {(optimal_retention['roe'] - baseline_roe)*100:.0f} bps over baseline")
    print(f"• Risk reduction: {(1 - optimal_retention['profit_volatility'] / (COMPANY_PARAMS['annual_revenue'] * COMPANY_PARAMS['base_operating_margin']))*100:.0f}% lower profit volatility")
else:
    print(f"• ROE improvement: [Run optimization for specific results]")
    print(f"• Risk reduction: [Run optimization for specific results]")

if 'median_final_with' in locals():
    print(f"• Long-term wealth: {median_final_with / COMPANY_PARAMS['initial_assets']:.1f}x initial assets after {n_years} years")
else:
    print(f"• Long-term wealth: [Run Section 6 for results]")

if 'survived_with' in locals():
    print(f"• Bankruptcy risk: <{(1-survived_with)*100:.0f}% over {n_years} years")
else:
    print(f"• Bankruptcy risk: [Run Section 6 for results]")

print("\n✅ IMPLEMENTATION NEXT STEPS:")
print("-" * 50)
print("1. Implement InsurancePricer in production environment")
print("2. Integrate with existing risk management systems") 
print("3. Set up market cycle monitoring dashboard")
print("4. Establish quarterly retention review process")
print("5. Train actuarial team on ergodic optimization principles")
print("6. Validate pricing models with historical loss data")

print("\n📊 KEY METRICS SUMMARY:")
print("-" * 50)

# Summary table of key metrics if available
if 'avg_annual_loss' in locals():
    print(f"Average Annual Loss: ${avg_annual_loss:,.0f}")
    
if 'hard_premium' in locals():
    print(f"Hard Market Premium: ${hard_premium:,.0f}")
    print(f"Normal Market Premium: ${pricing_results[MarketCycle.NORMAL]['total_premium']:,.0f}")
    print(f"Soft Market Premium: ${soft_premium:,.0f}")
    print(f"Market Variation: {actual_variation:.1f}%")
    
if 'gross_profit' in locals():
    print(f"Gross Annual Profit: ${gross_profit:,.0f}")
    if 'hard_premium' in locals():
        print(f"Premium as % of Profit (Normal): {pricing_results[MarketCycle.NORMAL]['total_premium']/gross_profit:.1%}")

print("\n" + "=" * 70)
print("END OF EXECUTIVE SUMMARY")

## Conclusion

This notebook demonstrates the power of dynamic insurance pricing with market cycle support. Key takeaways:

1. **Dynamic Pricing Matters**: Market cycles can cause premium variations of 30-50%, significantly impacting optimal retention decisions.

2. **Retention Optimization**: The optimal deductible varies by market cycle, with higher retentions preferred in hard markets.

3. **Ergodic Benefits**: Insurance provides positive time-average growth despite negative ensemble-average cost.

4. **Long-term Value**: Over multi-decade horizons, properly structured insurance programs enhance wealth accumulation and survival probability.

5. **Market Cycle Management**: Understanding and adapting to market cycles is crucial for cost-effective risk transfer.

The new insurance pricing module enables sophisticated retention optimization that accounts for both risk dynamics and market conditions, transforming insurance from a simple cost center to a strategic growth enabler.